# Install and downloads

### Load Huggingface API key
Here we use Huggingface models

Note: Please load a text file that contains your model api key to __Current Folder__.

>Name your file in either ***'api_key_huggingface.txt'***

In [1]:
%%bash
rm -r /content/SearchQuery2FuncCall
git clone https://github.com/XiaoLIUau/SearchQuery2FuncCall.git
pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


rm: cannot remove '/content/SearchQuery2FuncCall': No such file or directory
Cloning into 'SearchQuery2FuncCall'...


In [2]:
%%bash
pip install -r /content/SearchQuery2FuncCall/requirements.txt --quiet
pip install git+https://github.com/huggingface/peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 requires torchdata==0.7.0, but you have torchdata 0.5.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-req-build-u9ejccl2


# Setup

In [4]:
# Setups
from SearchQuery2FuncCall.load_datasets_n_login_HF import q2f_datasets
from SearchQuery2FuncCall.utilities_main import *
from SearchQuery2FuncCall.utilities_train_peft import *

Saved 340 examples to 'q2f_dataset.json'.
Separated 87 Search() examples to 'search_examples.json'.
Separated 253 non-Search() examples to 'non_search_examples.json'.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-3c31c33b5a4cd717/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 260
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 80
    })
})
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Load Base Model and setup model Configs

In [5]:
# Load model function
def load_model(model_name):
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True,
        bnb_8bit_use_double_quant=True,
        bnb_8bit_quant_type="nf4",
        bnb_8bit_compute_dtype=torch.bfloat16,
        llm_int8_enable_fp32_cpu_offload=True,
    )
    return AutoModelForSeq2SeqLM.from_pretrained(
                  model_name,
                  torch_dtype=torch.bfloat16, # need to check is this would work to reduce runtime memory
                  device_map='auto',
                  quantization_config=bnb_config,
                  )

# Load model directly
model_name = 'google/flan-t5-large'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = 'left'
original_model = load_model(model_name)


# Check trainable parameters
print('\n### Before freeze all parameters: ###')
print(print_number_of_trainable_model_parameters(original_model))
# Freeze model parameters
freeze_all_parameters(original_model)

(…)large/resolve/main/tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

(…)lan-t5-large/resolve/main/tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

(…)rge/resolve/main/special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

(…)e/flan-t5-large/resolve/main/config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at google/flan-t5-large and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(…)arge/resolve/main/generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]


### Before freeze all parameters: ###
trainable model parameters: 204336128
all model parameters: 783150080
percentage of trainable model parameters: 26.09%

### After freeze all parameters: ###
trainable model parameters: 0
all model parameters: 783150080
percentage of trainable model parameters: 0.00%


# Try with Base Model

In [6]:
example_idxs = [5, 9, 24, 78]#[78]#[49]#
generate_samples(example_idxs,q2f_datasets,original_model,tokenizer)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT:
What is -0.5 miles in kilometers
---------------------------------------------------------------------------------------------------
BASELINE OUTPUT:
UnitConvert(SourceUnit:mile,TargetUnit:kilometer,SourceValue:-0.5)
---------------------------------------------------------------------------------------------------
PEFT MODEL GENERATION - OUTPUT:
UnitConvert(-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,

---------------------------------------------------------------------------------------------------
Example  2
---------------------------------------------------------------------------------------------------
INPUT:
cosine of 60 degrees
------------------------------------------------------------

## PEFT training

In [7]:
# PEFT Setup
lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

# peft_model = original_model
peft_model = prepare_model_for_kbit_training(original_model)
peft_model = get_peft_model(original_model, lora_config)
print(print_number_of_trainable_model_parameters(peft_model))


########################
# Reduce memory
del original_model
n = gc.collect()
torch.cuda.empty_cache()
########################

output_dir = f'./peft-query-function-training-{str(int(time.time()))}'
peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=10,
    logging_steps=20,
    # max_steps=10
)


# reformat dataset for training by adding 'text' variable in examples
tokenized_datasets = tokenize_datasets(q2f_datasets,tokenizer)
print(q2f_datasets)

# # get max sequence length of the 'text' in training dataset
# max_seq_length = get_max_seq_length(q2f_datasets)

# # Set supervised fine-tuning parameters
# peft_trainer = SFTTrainer(
#     model=peft_model,
#     train_dataset=q2f_datasets['train'],
#     # eval_dataset=q2f_datasets['valid'],
#     peft_config=lora_config,
#     dataset_text_field="text",
#     max_seq_length=max_seq_length,
#     tokenizer=tokenizer,
#     args=peft_training_args,
#     packing=False,
# )


peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=20,
    logging_steps=30,
    max_steps=200
)
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)


## PEFT training
peft_trainer.train()

## Save PEFT Model
peft_model_path="./peft-query-function-checkpoint-local-seq2seq"
peft_trainer.model.save_pretrained(peft_model_path,
                                   local_files_only=True
                                   )
tokenizer.save_pretrained(peft_model_path,
                          local_files_only=True
                          )

trainable model parameters: 9437184
all model parameters: 792587264
percentage of trainable model parameters: 1.19%


Map:   0%|          | 0/260 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 260
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 80
    })
})


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
30,0.161300
60,0.010500
90,0.004900
120,0.004000
150,0.002900
180,0.002200


('./peft-query-function-checkpoint-local-seq2seq/tokenizer_config.json',
 './peft-query-function-checkpoint-local-seq2seq/special_tokens_map.json',
 './peft-query-function-checkpoint-local-seq2seq/tokenizer.json')

In [8]:
!ls -al ./peft-query-function-checkpoint-local-seq2seq/adapter_model.bin

-rw-r--r-- 1 root root 37854797 Oct 24 05:58 ./peft-query-function-checkpoint-local-seq2seq/adapter_model.bin


In [9]:
# !zip -r /content/SearchQuery2FuncCall/peft-query-function-checkpoint-local.zip /content/SearchQuery2FuncCall/peft-query-function-checkpoint-local
# from google.colab import files
# files.download("/content/SearchQuery2FuncCall/peft-query-function-checkpoint-local.zip")

In [10]:
!zip -r peft-query-function-checkpoint-local-seq2seq.zip ./peft-query-function-checkpoint-local-seq2seq
from google.colab import files
files.download("./peft-query-function-checkpoint-local-seq2seq.zip")

  adding: peft-query-function-checkpoint-local-seq2seq/ (stored 0%)
  adding: peft-query-function-checkpoint-local-seq2seq/adapter_model.bin (deflated 8%)
  adding: peft-query-function-checkpoint-local-seq2seq/special_tokens_map.json (deflated 86%)
  adding: peft-query-function-checkpoint-local-seq2seq/tokenizer_config.json (deflated 83%)
  adding: peft-query-function-checkpoint-local-seq2seq/adapter_config.json (deflated 46%)
  adding: peft-query-function-checkpoint-local-seq2seq/tokenizer.json (deflated 74%)
  adding: peft-query-function-checkpoint-local-seq2seq/README.md (deflated 65%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Load PEFT model  for inference
Require restart runtime due to memory issue

In [11]:
# import os
# os.kill(os.getpid(), 9)

del peft_model
del tokenizer
del peft_trainer
# del original_model

import gc
n = gc.collect()
torch.cuda.empty_cache()

In [12]:
n = gc.collect()
torch.cuda.empty_cache()

In [13]:
# # Setups
# from SearchQuery2FuncCall.load_datasets_n_login_HF import q2f_datasets
# from SearchQuery2FuncCall.utilities_main import *

In [14]:
### Load PEFT model from previous saved folder
# Note: It will only work if the colab session is not disconnected from previous training session
# Restart the runtime is fine

model_name = 'google/flan-t5-large'
peft_model_base = AutoModelForSeq2SeqLM.from_pretrained(model_name,
                                                      #  torch_dtype=torch.float16,
                                                       device_map="auto")
# peft_model_base = load_model(model_name)
peft_model_path="./peft-query-function-checkpoint-local-seq2seq"#"/content/SearchQuery2FuncCall/peft-query-function-checkpoint-local"#
# Load saved peft trained AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          # torch_dtype=torch.bfloat16,
                                          use_fast=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.truncation_side = "left"
# Load saved peft trained model
peft_model = PeftModel.from_pretrained(peft_model_base,
                                       peft_model_path,
                                       torch_dtype=torch.float16,
                                       device_map="auto",
                                       is_trainable=False
                                      )
print('\nBefore merge:')
print(print_number_of_trainable_model_parameters(peft_model))
peft_model = peft_model.merge_and_unload()
print('\nAfter merge:')
print(print_number_of_trainable_model_parameters(peft_model))

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at google/flan-t5-large and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Before merge:
trainable model parameters: 0
all model parameters: 792587264
percentage of trainable model parameters: 0.00%

After merge:
trainable model parameters: 0
all model parameters: 783150080
percentage of trainable model parameters: 0.00%


In [15]:
# import shutil
# # Get the path to the folder you want to remove
# folder_path = "/content/Untitled Folder"
# # Remove the folder recursively
# shutil.rmtree(folder_path)
!rm -r ./merged-model
merged_model_path="merged-model-seq2seq"
peft_model.save_pretrained(merged_model_path,local_files_only=True)
tokenizer.save_pretrained(merged_model_path,local_files_only=True)

rm: cannot remove './merged-model': No such file or directory


('merged-model-seq2seq/tokenizer_config.json',
 'merged-model-seq2seq/special_tokens_map.json',
 'merged-model-seq2seq/tokenizer.json')

In [39]:
del peft_model_base
del peft_model
del tokenizer

import gc
n = gc.collect()
torch.cuda.empty_cache()

In [40]:
n = gc.collect()
torch.cuda.empty_cache()

In [41]:
merged_model_path="merged-model-seq2seq"
# Load merged peft model in 4 bits
peft_model = AutoModelForSeq2SeqLM.from_pretrained(merged_model_path,
                                                  # device_map="auto",
                                                  # load_in_8bit=True,
                                                  # torch_dtype=torch.bfloat16
                                                  )
tokenizer = AutoTokenizer.from_pretrained(merged_model_path)#,torch_dtype=torch.bfloat16)

# Generate with selected examples

In [42]:
from datetime import datetime
# record current timestamp
start = datetime.now()
example_indices = [5, 9, 24, 78]#[49]#
generate_samples(example_indices,q2f_datasets,peft_model,tokenizer)
end = datetime.now()
# find difference loop start and end time and display
td = (end - start).total_seconds() * 10**3
print(f"The time of execution of above program is : {td:.03f}ms")

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT:
What is -0.5 miles in kilometers
---------------------------------------------------------------------------------------------------
BASELINE OUTPUT:
UnitConvert(SourceUnit:mile,TargetUnit:kilometer,SourceValue:-0.5)
---------------------------------------------------------------------------------------------------
PEFT MODEL GENERATION - OUTPUT:
UnitConvert(SourceUnit:mile,TargetUnit:kilometer,SourceValue:-0.5)

---------------------------------------------------------------------------------------------------
Example  2
---------------------------------------------------------------------------------------------------
INPUT:
cosine of 60 degrees
---------------------------------------------------------------------------------------------------
BASELINE OUTPUT:
Calculate

# Generate outputs for given test dataset
Here we are using the test dataset

In [43]:
index_s=0 # Start index for example in test dataset
df=generate_dataset(index_s,q2f_datasets,peft_model,tokenizer)
df

0
10
20
30
40
50
60
70


,Inputs,Outputs,Outputs_generated
0,2.3 * -4.5,Calculate(2.3*-4.5),Calculate(-2.3*-4.5)
1,15 + 7,Calculate(15+7),Calculate(15+7)
2,What is machine learning?,Search(),Search()
3,ounces to grams,"UnitConvert(SourceUnit:ounce,TargetUnit:gram,S...","UnitConvert(SourceUnit:ounce,TargetUnit:gram,S..."
4,log(0.1),Calculate(log(0.1)),Calculate(log(0.1))
...,...,...,...
75,10 / 2,Calculate(10/2),Calculate(10/2)
76,convert 90 celsius to fahrenheit,"UnitConvert(SourceUnit:celsius,TargetUnit:fahr...","UnitConvert(SourceUnit:celsius,TargetUnit:fahr..."
77,What is -4.5 times 2,Calculate(-4.5*2),Calculate(-4.5*2)
78,Evaluate 12 - 7,Calculate(12-7),Calculate(12-7)


# Evaluate using ROUGE and BLEU scores

In [44]:
evaluate_generations(df['Outputs'],df['Outputs_generated'])

MODEL ROUGE SCORES:
{'rouge1': 0.9415384615384614, 'rouge2': 0.5828514194139194, 'rougeL': 0.9341369047619046, 'rougeLsum': 0.9337355006105006}
MODEL BLEU SCORES:
{'bleu': 0.8542718058691469, 'precisions': [0.9289099526066351, 0.8770343580470162, 0.828752642706131, 0.7888040712468194], 'brevity_penalty': 1.0, 'length_ratio': 1.0160513643659712, 'translation_length': 633, 'reference_length': 623}


# Evaluation Results with flan-t5-large
___Merged model loaded in 8 bits use less than half the inference time compare to loaded in 32 bit float___
## Model loaded in 8 bits
MODEL ROUGE SCORES:
{'rouge1': 0.93760989010989, 'rouge2': 0.5732905636030636, 'rougeL': 0.9298511904761904, 'rougeLsum': 0.9297271825396827}

MODEL BLEU SCORES:
{'bleu': 0.8429376445805247, 'precisions': [0.9266347687400319, 0.870201096892139, 0.815845824411135, 0.7674418604651163], 'brevity_penalty': 1.0, 'length_ratio': 1.0064205457463884, 'translation_length': 627, 'reference_length': 623}

## Model loaded normally in 32 bits float
MODEL ROUGE SCORES:
{'rouge1': 0.9415384615384614, 'rouge2': 0.5828514194139194, 'rougeL': 0.9341369047619046, 'rougeLsum': 0.9337355006105006}

MODEL BLEU SCORES:
{'bleu': 0.8542718058691469, 'precisions': [0.9289099526066351, 0.8770343580470162, 0.828752642706131, 0.7888040712468194], 'brevity_penalty': 1.0, 'length_ratio': 1.0160513643659712, 'translation_length': 633, 'reference_length': 623}

# Evaluation Results with flan-t5-large

**Merged model loaded in 8 bits use less than half the inference time compare to loaded in 32 bit float**

| Metric                 | Model loaded in 8 bits | Model loaded in 32 bits float |
|------------------------|------------------------|-------------------------------|
| Rouge-1 Score          | 0.9376                 | 0.9415                        |
| Rouge-2 Score          | 0.5733                 | 0.5829                        |
| Rouge-L Score          | 0.9299                 | 0.9341                        |
| Rouge-Lsum Score       | 0.9297                 | 0.9337                        |

| Metric                 | Model loaded in 8 bits | Model loaded in 32 bits float |
|------------------------|------------------------|-------------------------------|
| BLEU Score             | 0.8429                 | 0.8543                        |
| BLEU Precisions        | [0.9266, 0.8702, 0.8158, 0.7674] | [0.9289, 0.8770, 0.8288, 0.7888] |
| Brevity Penalty        | 1.0                    | 1.0                           |
| Length Ratio           | 1.0064                 | 1.0161                        |
| Translation Length     | 627                    | 633                           |
| Reference Length       | 623                    | 623                           |


## Evaluation Results with flan-t5-large
MODEL ROUGE SCORES:
{'rouge1': 0.9415384615384614, 'rouge2': 0.5828514194139194, 'rougeL': 0.9341369047619046, 'rougeLsum': 0.9337355006105006}

MODEL BLEU SCORES:
{'bleu': 0.8542718058691469, 'precisions': [0.9289099526066351, 0.8770343580470162, 0.828752642706131, 0.7888040712468194], 'brevity_penalty': 1.0, 'length_ratio': 1.0160513643659712, 'translation_length': 633, 'reference_length': 623}

## Evaluation Results with quantized LLama 2
MODEL ROUGE SCORES:
{'rouge1': 0.9827777777777779, 'rouge2': 0.7005208333333333, 'rougeL': 0.9833333333333334, 'rougeLsum': 0.9833333333333332}

MODEL BLEU SCORES:
{'bleu': 0.9334276042273965, 'precisions': [0.9897660818713451, 0.9751655629139073, 0.9580152671755725, 0.9391891891891891], 'brevity_penalty': 0.9669333300994326, 'length_ratio': 0.9674681753889675, 'translation_length': 684, 'reference_length': 707}

| Model                                  | Rouge-1     | Rouge-2     | Rouge-L     | Rouge-Lsum  | Bleu       | Precision (P1) | Precision (P2) | Precision (P3) | Precision (P4) | Brevity Penalty | Length Ratio | Translation Length | Reference Length |
|----------------------------------------|------------|------------|------------|------------|-----------|----------------|----------------|----------------|----------------|-----------------|--------------|-------------------|------------------|
| quantized LLama 2 fine-tuned with peft | 0.9828     | 0.7005     | 0.9833     | 0.9833     | 0.9334    | 0.9898         | 0.9752         | 0.9580         | 0.9392         | 0.9669          | 0.9675       | 684               | 707              |
| Palm API with prompt engineering       | 0.9792     | 0.6571     | 0.9787     | 0.9797     | 0.9031    | 0.9669         | 0.9402         | 0.9049         | 0.8588         | 0.9851          | 0.9852       | 665               | 675              |
| flan-t5-large fine-tuned with peft     | 0.9415     | 0.5829     | 0.9341     | 0.9337     | 0.8543    | 0.9289         | 0.8770         | 0.8288         | 0.7888         | 1.0             | 1.0161       | 633               | 623              |
| quantized LLama 2 with prompt engineering | 0.8859  | 0.5330     | 0.8777     | 0.8810     | 0.8039    | 0.9368         | 0.8771         | 0.8297         | 0.7942         | 0.9372          | 0.9391       | 617               | 657              |
